In [46]:
import pandas as pd
import numpy as np 
import re
import seaborn as sns
import matplotlib.pyplot as plt
import geopandas as gpd
from pathlib import Path
from shapely.ops import transform
from pyproj import Transformer


## Extreme

In [80]:
df_extreme_data = pd.read_csv('../notebooks/pct_ones_extreme_risk_by_county.csv', dtype={'PERSON ID': str}) 

In [81]:
df_extreme_data.head()

,County,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,Anger Control,Anxiety,...,Mental_Health_Psychotic_Thought_Disturbances,Mental_Health_Risk_Safety,Mental_Health_Sexual_Developmental_Concerns,Mental_Health_Trauma_Stress_Related,Supportive_Adult_Relationships_Family_Relationships,Supportive_Adult_Relationships_Social_Relationships,Supportive_Adult_Relationships_Barriers_to_Support,Housing_Instability_Skills_Readiness,Housing_Instability_Current_Living_Situation,Housing_Instability_Barriers_Risks
0,Anderson,1.83,0.92,2.75,0.92,46.79,0.92,5.62,2.75,0.0,...,0.0,11.54,2.67,70.59,90.00,98.46,81.82,8.70,14.71,26.47
1,Bedford,2.00,0.00,0.00,0.00,44.00,0.00,0.00,2.00,0.0,...,0.0,0.00,2.78,30.00,71.43,97.06,55.56,17.65,15.38,11.11
2,Benton,23.08,0.00,0.00,0.00,30.77,0.00,0.00,0.00,0.0,...,0.0,NaN,0.00,100.00,100.00,100.00,NaN,0.00,60.00,0.00
3,Bledsoe,0.00,0.00,0.00,0.00,33.33,0.00,0.00,6.67,0.0,...,0.0,0.00,0.00,20.00,0.00,77.78,33.33,0.00,25.00,0.00
4,Blount,2.22,1.11,2.22,0.00,48.89,0.00,0.00,1.11,0.0,...,0.0,33.33,4.26,88.24,100.00,97.78,88.24,3.70,45.83,24.14


In [82]:
# --- Load TN counties from TIGER/Line ---
shp = Path('../data/bdaic created tables/tl_2024_us_county/tl_2024_us_county.shp')
gdf = gpd.read_file(shp)
gdf = gdf[gdf['STATEFP'] == '47'].copy()

In [83]:
# Load TN counties base geometry (95 rows)
base = gpd.read_file("../data/bdaic created tables/tl_2024_us_county/tl_2024_us_county.shp")
tn = base[base["STATEFP"] == "47"][["GEOID","NAME","geometry"]].copy()
tn["GEOID"] = tn["GEOID"].astype(str).str.zfill(5)

In [63]:
tn.head()

,GEOID,NAME,geometry
29,47065,Hamilton,"POLYGON ((-85.17523 34.98598, -85.17663 34.986..."
64,47115,Marion,"POLYGON ((-85.87302 34.99119, -85.87301 34.991..."
67,47185,White,"MULTIPOLYGON (((-85.6446 36.01505, -85.64392 3..."
127,47129,Morgan,"POLYGON ((-84.45834 36.18654, -84.45834 36.186..."
180,47013,Campbell,"POLYGON ((-84.09196 36.22261, -84.09202 36.222..."


In [84]:
df_extreme_data.head()

,County,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,Anger Control,Anxiety,...,Mental_Health_Psychotic_Thought_Disturbances,Mental_Health_Risk_Safety,Mental_Health_Sexual_Developmental_Concerns,Mental_Health_Trauma_Stress_Related,Supportive_Adult_Relationships_Family_Relationships,Supportive_Adult_Relationships_Social_Relationships,Supportive_Adult_Relationships_Barriers_to_Support,Housing_Instability_Skills_Readiness,Housing_Instability_Current_Living_Situation,Housing_Instability_Barriers_Risks
0,Anderson,1.83,0.92,2.75,0.92,46.79,0.92,5.62,2.75,0.0,...,0.0,11.54,2.67,70.59,90.00,98.46,81.82,8.70,14.71,26.47
1,Bedford,2.00,0.00,0.00,0.00,44.00,0.00,0.00,2.00,0.0,...,0.0,0.00,2.78,30.00,71.43,97.06,55.56,17.65,15.38,11.11
2,Benton,23.08,0.00,0.00,0.00,30.77,0.00,0.00,0.00,0.0,...,0.0,NaN,0.00,100.00,100.00,100.00,NaN,0.00,60.00,0.00
3,Bledsoe,0.00,0.00,0.00,0.00,33.33,0.00,0.00,6.67,0.0,...,0.0,0.00,0.00,20.00,0.00,77.78,33.33,0.00,25.00,0.00
4,Blount,2.22,1.11,2.22,0.00,48.89,0.00,0.00,1.11,0.0,...,0.0,33.33,4.26,88.24,100.00,97.78,88.24,3.70,45.83,24.14


In [87]:
tn.head()

,GEOID,NAME,geometry
29,47065,Hamilton,"POLYGON ((-85.17523 34.98598, -85.17663 34.986..."
64,47115,Marion,"POLYGON ((-85.87302 34.99119, -85.87301 34.991..."
67,47185,White,"MULTIPOLYGON (((-85.6446 36.01505, -85.64392 3..."
127,47129,Morgan,"POLYGON ((-84.45834 36.18654, -84.45834 36.186..."
180,47013,Campbell,"POLYGON ((-84.09196 36.22261, -84.09202 36.222..."


In [88]:
tn.rename(columns={"NAME": "County"}, inplace=True)

In [89]:
# Merge attributes onto the 95 TN polygons
gdf_merged = tn.merge(df_extreme_data, on="County", how="left")
print(len(gdf_merged))  # should be 95

95


In [90]:
gdf_merged.head()

,GEOID,County,geometry,Decision-Making (Judgement),Impulsivity/Hyperactivity,School Behavior,Substance Use,Spiritual/Religious,Traditions And Rituals,Youth Involvement With Care,...,Mental_Health_Psychotic_Thought_Disturbances,Mental_Health_Risk_Safety,Mental_Health_Sexual_Developmental_Concerns,Mental_Health_Trauma_Stress_Related,Supportive_Adult_Relationships_Family_Relationships,Supportive_Adult_Relationships_Social_Relationships,Supportive_Adult_Relationships_Barriers_to_Support,Housing_Instability_Skills_Readiness,Housing_Instability_Current_Living_Situation,Housing_Instability_Barriers_Risks
0,47065,Hamilton,"POLYGON ((-85.17523 34.98598, -85.17663 34.986...",6.75,0.40,3.57,5.16,53.57,0.0,4.29,...,0.0,16.22,1.53,42.11,77.78,96.09,53.49,3.03,25.26,12.82
1,47115,Marion,"POLYGON ((-85.87302 34.99119, -85.87301 34.991...",0.00,0.00,0.00,0.00,33.33,0.0,0.00,...,0.0,0.00,0.00,NaN,100.00,100.00,100.00,0.00,100.00,50.00
2,47185,White,"MULTIPOLYGON (((-85.6446 36.01505, -85.64392 3...",2.63,2.63,0.00,0.00,39.47,0.0,3.03,...,0.0,11.11,4.00,0.00,100.00,100.00,100.00,0.00,21.43,0.00
3,47129,Morgan,"POLYGON ((-84.45834 36.18654, -84.45834 36.186...",9.09,9.09,9.09,0.00,27.27,0.0,0.00,...,0.0,0.00,0.00,100.00,75.00,100.00,66.67,33.33,57.14,25.00
4,47013,Campbell,"POLYGON ((-84.09196 36.22261, -84.09202 36.222...",3.92,0.00,3.92,0.00,49.02,0.0,3.57,...,0.0,11.11,2.38,40.00,90.00,100.00,91.67,12.50,17.65,16.67


In [91]:
len(gdf_merged)

95

In [92]:
gdf_merged.to_csv('extreme_risk_mapping.csv', index=False)

## Risk

In [75]:
df_risk_data = pd.read_csv('../notebooks/risk_by_county_mapping.csv', dtype={'PERSON ID': str}) 

In [76]:
df_risk_data = df_risk_data.rename(columns={"County": "NAME"})

In [77]:
# Merge attributes onto the 95 TN polygons
gdf_merged_risk = tn.merge(df_risk_data, on="NAME", how="left")
print(len(gdf_merged_risk))  # should be 95

95


In [78]:
gdf_merged_risk.to_csv('risk_mapping.csv', index=False)

## ARCGIS

#### Extreme

In [74]:
gdf_4326 = gdf_merged.to_crs(epsg=4326)
gdf_4326.to_file("extreme_risk_joined.geojson", driver="GeoJSON")

### Risk

In [79]:
gdf_4326 = gdf_merged_risk.to_crs(epsg=4326)
gdf_4326.to_file("risk_joined.geojson", driver="GeoJSON")